# Heat Geodesics en NIFs

Tenemos la PDE 
$$\frac{\partial u}{\partial t} = \nabla \cdot ( P_{\nabla \psi} \nabla u) = \Delta u - (\nabla u \cdot \nabla \psi) \Delta \psi$$

Definimos $u_0(\vec{x}) = e^{-\varepsilon \left || \vec{x} - \vec{p} | \right |^2}$. Entonces,

$$\nabla u_0(\vec{x}) = -2 \varepsilon u_0(\vec{x}) (\vec{x} - \vec{p}) \quad \quad \text{y} \quad \quad \Delta u_0(\vec{x}) = 2\varepsilon \, u_0(\vec{x}) \, (2 \varepsilon \left || \vec{x} - \vec{p} | \right |^2 - 3) $$

### Forward Euler
Este esquema es mas sencillo (pero mas inestable), integramos con respecto al tiempo tomando la siguiente aproximacion:
$$u_t(\vec{x}) = u_0(\vec{x}) + t (\Delta u_0(\vec{x}) - (\nabla u_0 \cdot \nabla \psi) \Delta \psi)$$

### Backward Euler
Integrador implicito:
$$u_t = u_0 + t (\nabla \cdot \circ \, P_{\nabla \psi} \nabla) u_t$$

Donde se entiende por $(\nabla \cdot \circ \, P_{\nabla \psi}) : C^k \rightarrow C^k$ el operador que actua sobre funciones $u$, el cual aplica la proyeccion del gradiente sobre el espacio ortogonal a $\nabla \psi$ y luego la divergencia.

$$t(id - \nabla \cdot \circ \, P_{\nabla \psi} \nabla) u_t = u_0$$

Entonces, $u_t$ es la funcion que minimiza la energia:

$$ \underset{u: \Omega \rightarrow \mathbb{R}}{\text{min}} \quad \quad \int_\Omega \left | t(id - \nabla \cdot \circ \, P_{\nabla \psi} \,\nabla) u_t - u_0 \right | \quad d\vec{x}.$$

Tengo muchas formas de realizar esta optimizacion:
1. CML dada una familia de funciones
2. Discretizacion y FEM ?
3. Usando el mismo SIREN

Hay que tener fe de que estas minimizaciones ganen ventaja frente a FE... ta dificil.

In [2]:
import numpy as np
import torch
from src.model import SIREN
from src.obj import load
import trimesh as tm
import meshplot as mp
import src.diff_operators as dif

In [ ]:
#mesh = tm.load_mesh('results/bunny/experiment/reconstructions/mc_mesh_best.obj')

#plot = mp.plot( mesh.vertices, mesh.faces, return_plot=True )
#plot.add_points( p, shading={'point_size':0.2})

In [3]:
model = SIREN(
        n_in_features= 3,
        n_out_features=1,
        hidden_layer_config=[512]*4,
        w0=30,
        ww=None,
        activation= 'sine'
)
model.load_state_dict( torch.load('results/bunny/experiment/models/model_best.pth', weights_only=True))

device_torch = torch.device(0)
model.to(device_torch)

/home/mfainstein/miniconda3/envs/tdf/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


SIREN(
  (net): Sequential(
    (0): Sequential(
      (0): Linear(in_features=3, out_features=512, bias=True)
      (1): SineLayer(w0=30)
    )
    (1): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): SineLayer(w0=30)
    )
    (2): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): SineLayer(w0=30)
    )
    (3): Sequential(
      (0): Linear(in_features=512, out_features=512, bias=True)
      (1): SineLayer(w0=30)
    )
    (4): Sequential(
      (0): Linear(in_features=512, out_features=1, bias=True)
    )
  )
)

In [16]:
def grad_desc( model, samples, max_batch=64**2, device=torch.device(0), iterations = 3 ):
    # samples = ( amount_samples, 3 )
    amount_samples = samples.shape[0]

    head = 0
    X = samples.copy()

    for i in range( iterations):
        print(f'Iteration: {i}')
        mean_distance = 0
        while head < amount_samples:
            
            inputs_subset = torch.from_numpy( X[head:min(head + max_batch, amount_samples), :] ).to(device).unsqueeze(0).float()

            x, y = model(inputs_subset).values()

            mean_distance += torch.sum(y).detach().cpu().numpy()

            grad_psi = torch.nn.functional.normalize( dif.gradient(y,x), dim=-1 )

            X[head:min(head + max_batch, amount_samples)] -= (y *  grad_psi).squeeze(0).detach().cpu().numpy()

            head += max_batch

        print(f'Mean distance: { mean_distance / amount_samples }')

    return X

In [50]:
EPSILON = 1e-3
pc, _ = load( 'data/bunny/bunny_pc.obj' )

pc = grad_desc( model, pc, iterations=1 )

u0 = lambda x, p:  (torch.exp( -EPSILON * torch.sum( (x - p)**2, dim=-1) ))[...,None]
grad_u0 = lambda x, p: -2 * EPSILON * u0(x, p) * (x - p)
lap_u0 = lambda x, p: ( 2 * EPSILON * u0(x, p) * ( 2* EPSILON * torch.sum( (x - p)**2, dim=-1) - 3 )[...,None])

t = 1e-4

Iteration: 0
Mean distance: -9.786325991153717e-05


In [71]:
def project( projectees, projectors ):
    # asumiendo ||projectors|| = 1
    return projectees -  torch.sum(projectors * projectees, dim=-1)[...,None] * projectors

def computeX( model, samples, p, u0, grad_u0, max_batch=64**2, device=torch.device(0) ):
    # samples = ( amount_samples, 3 )    
    head = 0
    amount_samples = samples.shape[0]

    uts = np.zeros( (amount_samples, 1))
    X = np.zeros( (amount_samples, 3))
    divX = np.zeros( (amount_samples, 1))

    ps = torch.from_numpy( np.tile(p, (max_batch, 1)) ).to(device_torch)
    i = 0
    while head < amount_samples:
        print(f'Iteration: {i}')
        
        inputs_subset = torch.from_numpy( samples[head:min(head + max_batch, amount_samples), :] ).to(device).unsqueeze(0).float()

        x, y =  model(inputs_subset).values()

        grad_psi = dif.gradient(y,x)

        ps_ss = ps[:inputs_subset.shape[1],:].unsqueeze(0)

        u0s = u0( inputs_subset, ps_ss )

        # RK4: ( out of memory... lo mata hacer el gradiente de u para calcular X.. es un monton)
        #k1 = dif.divergence( project( grad_u0( inputs_subset, ps_ss ), grad_psi ) , x )
        #k2 = dif.divergence( project( dif.gradient(u0s + 0.5 * t * k1 , x), grad_psi ), x )
        #k3 = dif.divergence( project( dif.gradient(u0s + 0.5 * t * k2 , x), grad_psi ), x )
        #k4 = dif.divergence( project( dif.gradient(u0s + t * k3 , x), grad_psi ), x )
        #ut = u0( inputs_subset, ps_ss ) + (t/6) * (k1 + 2*k2 + 2*k3 + k4)

        # Heuns:
        k1 = dif.divergence( project( grad_u0( inputs_subset, ps_ss ), grad_psi ) , x )
        k2 = dif.divergence( project( dif.gradient(u0s + t * k1 , x), grad_psi ), x )
        ut = u0( inputs_subset, ps_ss ) + (t/2) * (k1 + k2)
        
        uts[head:min(head + max_batch, amount_samples)] = ut.detach().cpu().squeeze(0).numpy()

        #X_ss = -1 * torch.nn.functional.normalize( project( dif.gradient( ut, x ), grad_psi ) , dim=-1 )
        X_ss = -1 * torch.nn.functional.normalize(project( grad_u0( inputs_subset, ps_ss ), grad_psi ) , dim=-1 ) #-1 * torch.nn.functional.normalize(  dif.gradient( ut, x ) , dim=-1 )
        #X_ss = -1 * project( dif.gradient( ut, x ), grad_psi )

        X[head:min(head + max_batch, amount_samples)] = X_ss.detach().cpu().squeeze(0).numpy()
        #divX[head:min(head + max_batch, amount_samples)] = dif.divergence( X_ss, x ).detach().cpu().squeeze(0).numpy()

        head += max_batch
        i += 1

        torch.cuda.empty_cache()

    return uts, X, divX

ut, X, divX = computeX( model, pc, pc[10,:], u0, grad_u0, max_batch=64**2 )

Iteration: 0
Iteration: 1
Iteration: 2


In [72]:
np.min(ut), np.max(ut)

(0.9983037662802133, 1.0000000000000029)

In [74]:
plot = mp.plot( pc, c= ut, shading={'point_size':0.05, 'v_min':0, 'v_max':1}, return_plot=True )
plot.add_points( pc[10,:][None,...], shading={'point_size':0.5})
plot.add_lines( pc, pc + X * 0.050 )

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.1099628…

2

In [54]:
u0_numpy = lambda x, p:  (np.exp( -EPSILON * np.sum( (x - p)**2, axis=-1) ))[...,None]
grad_u0_numpy = lambda x, p: -2 * EPSILON * u0_numpy(x, p) * (x - p)

u0s = u0_numpy( pc,np.tile(pc[10,:], (len(pc), 1)) )
plot = mp.plot( pc, c= u0s, shading={'point_size':0.05, 'v_min':0, 'v_max':1}, return_plot=True )
plot.add_points( pc[10,:][None,...], shading={'point_size':0.5})
#plot.add_lines( pc, pc + X * 0.05 )

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.1099628…

1

In [55]:
np.mean( np.abs( u0s - ut ) )

3.5860205329482086e-07